### Import Required Libraries and Set Up Environment Variables

In [25]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [ ]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")



### Access the New York Times API

In [27]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
query_url = f"{url}api-key={nyt_api_key}&begin_date={begin_date}&end_date={end_date}&fq={filter_query}&sort={sort}&fl={field_list}"


In [28]:
# Create an empty list to store the reviews
star_review = []

# loop through pages 0-19
for page in range(0,19):
    page_query_url = f"{query_url}&page={page}"
    response = requests.get(page_query_url)
    reviews = response.json()
    

    
    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)
    
    # Try and save the reviews to the reviews_list
    try:

        # loop through the reviews["response"]["docs"] and append each review to the list
        for review in reviews["response"]["docs"]:
            star_review.append(review)

        # Print the page that was just retrieved
        print ("page=",page)
    except:        

        # Print the page number that had no results then break from the loop
        print ("page with no results",page)
        break



page= 0
page= 1
page= 2
page= 3
page= 4
page= 5
page= 6
page= 7
page= 8
page= 9
page= 10
page= 11
page= 12
page= 13
page= 14
page= 15
page= 16
page= 17
page= 18


In [29]:
# Preview the first 5 results in JSON format
print(json.dumps(star_review[:5],indent=4))
# Use json.dumps with argument indent=4 to format data


[
    {
        "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
        "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
        "source": "The New York Times",
        "headline": {
            "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
            "kicker": null,
            "content_kicker": null,
            "print_headline": "The Attachment Diaries",
            "name": null,
            "seo": null,
            "sub": null
        },
        "keywords": [
            {
                "name": "subject",
                "value": "Movies",
                "rank": 1,
                "major": "N"
            },
            {
                "name": "creative_works",
                "value": "The Attachment Diaries (Movie)",
                "rank": 2,
                "major": "N"
            },
            {
                "name": "persons",
 

In [30]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
reviews_df = pd.json_normalize(star_review)


In [31]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early
reviews_df["title"] = reviews_df["headline.main"].apply(lambda st: st[st.find("\u2018")+1:st.find("\u2019 Review")])
reviews_df.head()

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What’s Love Got to Do With It?’ Probably a Lo
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children


In [32]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
reviews_df["keywords"] = reviews_df["keywords"].apply(extract_keywords)


In [33]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
titles = reviews_df["title"].tolist()
titles

['The Attachment Diaries',
 'What’s Love Got to Do With It?’ Probably a Lo',
 'You Can Live Forever',
 'A Tourist’s Guide to Love',
 'Other People’s Children',
 'One True Loves',
 'The Lost Weekend: A Love Story',
 'A Thousand and One',
 'Your Place or Mine',
 'Love in the Time of Fentanyl',
 'Pamela, a Love Story',
 'In From the Side',
 'After Love',
 'Alcarràs',
 'Nelly & Nadine',
 'Lady Chatterley’s Lover',
 'The Sound of Christmas',
 'The Inspection',
 'Bones and All',
 'My Policeman',
 'About Fate',
 'Waiting for Bojangles',
 'I Love My Dad',
 'A Love Song',
 'Alone Together',
 'Art of Love',
 'The Wheel',
 'Thor: Love and Thunder',
 'Both Sides of the Blade',
 'Fire of Love',
 'Love & Gelato',
 'Stay Prayed Up',
 'Benediction',
 'Dinner in America',
 'In a New York Minute',
 'Anaïs in Love',
 'I Love America',
 'See You Then',
 'La Mami',
 'Love After Love',
 'Deep Water',
 'Lucy and Desi',
 'Cyrano',
 'The In Between',
 'Book of Love',
 'Lingui, the Sacred Bonds',
 'The Pink Clo

### Access The Movie Database API

In [34]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key

In [35]:
# Create an empty list to store the results


# Create a request counter to sleep the requests after a multiple
# of 50 requests
tmdb_movies_list = []
request_counter = 1


# Loop through the titles
for title in titles:
    if request_counter % 50 == 0:
        print("Sleeping for a second...")
        time.sleep(1)
    request_counter += 1
    query_url = f"{url}{title}{tmdb_key_string}"
    response = requests.get(query_url)
    movie_data = response.json()

    # Check if we need to sleep before making a request


    # Add 1 to the request counter



    
    # Perform a "GET" request for The Movie Database


    # Include a try clause to search for the full movie details.
        
    try:
        movie_id = movie_data["results"][0]["id"]
        movie_url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb_api_key}"
        movie_details = requests.get(movie_url).json()
        
        genres = [genre['name'] for genre in movie_details['genres']]
        spoken_languages = [lang['english_name'] for lang in movie_details['spoken_languages']]
        production_countries = [country['name'] for country in movie_details['production_countries']]
        
        movie_dict = {
            "title": title,
            "original_title": movie_details.get('original_title', ''),
            "budget": movie_details.get('budget', 0),
            "original_language": movie_details.get('original_language', ''),
            "homepage": movie_details.get('homepage', ''),
            "overview": movie_details.get('overview', ''),
            "popularity": movie_details.get('popularity', 0),
            "runtime": movie_details.get('runtime', 0),
            "revenue": movie_details.get('revenue', 0),
            "release_date": movie_details.get('release_date', ''),
            "vote_average": movie_details.get('vote_average', 0),
            "vote_count": movie_details.get('vote_count', 0),
            "genres": genres,
            "spoken_languages": spoken_languages,
            "production_countries": production_countries}
   


        # Add the relevant data to a dictionary and
        # append it to the tmdb_movies_list list
    
        tmdb_movies_list.append(movie_dict)
        print(f"Movie '{title}' found and added.")
    
    except IndexError:
        print(f"Movie '{title}' not found.")

        
        # Print out the title that was found



Movie 'The Attachment Diaries' found and added.
Movie 'What’s Love Got to Do With It?’ Probably a Lo' not found.
Movie 'You Can Live Forever' found and added.
Movie 'A Tourist’s Guide to Love' found and added.
Movie 'Other People’s Children' found and added.
Movie 'One True Loves' found and added.
Movie 'The Lost Weekend: A Love Story' found and added.
Movie 'A Thousand and One' found and added.
Movie 'Your Place or Mine' found and added.
Movie 'Love in the Time of Fentanyl' found and added.
Movie 'Pamela, a Love Story' found and added.
Movie 'In From the Side' found and added.
Movie 'After Love' found and added.
Movie 'Alcarràs' found and added.
Movie 'Nelly & Nadine' found and added.
Movie 'Lady Chatterley’s Lover' found and added.
Movie 'The Sound of Christmas' found and added.
Movie 'The Inspection' found and added.
Movie 'Bones and All' found and added.
Movie 'My Policeman' found and added.
Movie 'About Fate' found and added.
Movie 'Waiting for Bojangles' found and added.
Movie 'I

In [36]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(tmdb_movies_list[:5], indent=4))



[
    {
        "title": "The Attachment Diaries",
        "original_title": "El apego",
        "budget": 0,
        "original_language": "es",
        "homepage": "",
        "overview": "Argentina, 1970s. A desperate young woman goes to a clinic to have a clandestine abortion. As her pregnancy is already through the fourth month, the doctor refuses. Instead, she proposes to sell the baby to one of her clients and offers to provide shelter in her house until the child is born. Their disturbed personalities will become intertwined in a strange and dangerous relationship.",
        "popularity": 1.105,
        "runtime": 102,
        "revenue": 0,
        "release_date": "2021-10-07",
        "vote_average": 3.0,
        "vote_count": 4,
        "genres": [
            "Drama",
            "Mystery",
            "Thriller",
            "Horror"
        ],
        "spoken_languages": [
            "Spanish"
        ],
        "production_countries": [
            "Argentina"
        ]
 

In [37]:
# Convert the results to a DataFrame
tmdb_df = pd.DataFrame(tmdb_movies_list)

tmdb_df.head()


,title,original_title,budget,original_language,homepage,overview,popularity,runtime,revenue,release_date,vote_average,vote_count,genres,spoken_languages,production_countries
0,The Attachment Diaries,El apego,0,es,,"Argentina, 1970s. A desperate young woman goes...",1.105,102,0,2021-10-07,3.000,4,"[Drama, Mystery, Thriller, Horror]",[Spanish],[Argentina]
1,You Can Live Forever,You Can Live Forever,0,en,https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",27.274,96,15055,2023-03-24,6.500,43,"[Drama, Romance]","[English, French]","[Canada, United States of America]"
2,A Tourist’s Guide to Love,A Tourist's Guide to Love,0,en,https://www.netflix.com/title/81424906,"After an unexpected break up, a travel executi...",13.153,96,0,2023-04-21,6.300,172,"[Romance, Comedy]","[English, Vietnamese]",[United States of America]
3,Other People’s Children,Les Enfants des autres,0,fr,https://www.wildbunch.biz/movie/other-peoples-...,"Rachel loves her life, her students, her frien...",7.386,104,84178,2022-09-21,6.808,200,"[Drama, Comedy]","[French, English]",[France]
4,One True Loves,One True Loves,0,en,,Emma and Jesse are living the perfect life tog...,12.731,100,37820,2023-04-07,6.400,85,"[Romance, Comedy, Drama]","[English, Spanish]","[Czech Republic, United States of America]"


### Merge and Clean the Data for Export

In [38]:
# Merge the New York Times reviews and TMDB DataFrames on title
merged_df = pd.merge(reviews_df, tmdb_df, on="title")
merged_df.head()

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,...,overview,popularity,runtime,revenue,release_date,vote_average,vote_count,genres,spoken_languages,production_countries
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,"Argentina, 1970s. A desperate young woman goes...",1.105,102,0,2021-10-07,3.000,4,"[Drama, Mystery, Thriller, Horror]",[Spanish],[Argentina]
1,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,...,"When Jaime, a gay teenager, is sent to live in...",27.274,96,15055,2023-03-24,6.500,43,"[Drama, Romance]","[English, French]","[Canada, United States of America]"
2,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,subject: Movies;creative_works: A Tourist's Gu...,2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,...,"After an unexpected break up, a travel executi...",13.153,96,0,2023-04-21,6.300,172,"[Romance, Comedy]","[English, Vietnamese]",[United States of America]
3,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"subject: Movies;persons: Zlotowski, Rebecca;cr...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,...,"Rachel loves her life, her students, her frien...",7.386,104,84178,2022-09-21,6.808,200,"[Drama, Comedy]","[French, English]",[France]
4,https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"subject: Movies;persons: Bracey, Luke (1989- )...",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,...,Emma and Jesse are living the perfect life tog...,12.731,100,37820,2023-04-07,6.400,85,"[Romance, Comedy, Drama]","[English, Spanish]","[Czech Republic, United States of America]"


In [39]:
# Remove list brackets and quotation marks on the columns containing lists
columns_to_fix = ["genres", "spoken_languages", "production_countries"]
characters_to_remove = ["[", "]", "'"]

# Create a list of the columns that need fixing


# Create a list of characters to remove


# Loop through the list of columns to fix
for col in columns_to_fix:
    merged_df[col] = merged_df[col].astype("str")
    for char in characters_to_remove:
        merged_df[col] = merged_df[col].str.replace(char, '')

merged_df.head()


    # Convert the column to type 'str'


    # Loop through characters to remove


# Display the fixed DataFrame


,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,...,overview,popularity,runtime,revenue,release_date,vote_average,vote_count,genres,spoken_languages,production_countries
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,"Argentina, 1970s. A desperate young woman goes...",1.105,102,0,2021-10-07,3.000,4,"Drama, Mystery, Thriller, Horror",Spanish,Argentina
1,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,...,"When Jaime, a gay teenager, is sent to live in...",27.274,96,15055,2023-03-24,6.500,43,"Drama, Romance","English, French","Canada, United States of America"
2,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,subject: Movies;creative_works: A Tourist's Gu...,2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,...,"After an unexpected break up, a travel executi...",13.153,96,0,2023-04-21,6.300,172,"Romance, Comedy","English, Vietnamese",United States of America
3,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"subject: Movies;persons: Zlotowski, Rebecca;cr...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,...,"Rachel loves her life, her students, her frien...",7.386,104,84178,2022-09-21,6.808,200,"Drama, Comedy","French, English",France
4,https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"subject: Movies;persons: Bracey, Luke (1989- )...",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,...,Emma and Jesse are living the perfect life tog...,12.731,100,37820,2023-04-07,6.400,85,"Romance, Comedy, Drama","English, Spanish","Czech Republic, United States of America"


In [40]:
# Drop "byline.person" column
merged_df = merged_df.drop(columns="byline.person")

In [41]:
# Delete duplicate rows and reset index
merged_df = merged_df.drop_duplicates().reset_index(drop=True)


In [42]:
# Export data to CSV without the index
merged_df.to_csv("merged_movie_data.csv", index=False)